In [2]:
import pandas as pd
import numpy as np
from cleaning import ingest_data
from cleaning import get_sheet_data
from cleaning import SHEET_NAMES
from cleaning import clean


In [3]:
dict_data = ingest_data(r'C:\Users\USER\Documents\Project_setup\Food_Pricing_Prediction\data\SELECTED FOOD (JAN_2019 - NOV 2021).csv', SHEET_NAMES)

In [4]:
dollar_price = pd.read_excel(r'C:\Users\USER\Documents\Project_setup\Food_Pricing_Prediction\data\Dollar_Prices.xlsx')

In [5]:
dollar_price['Date'] = pd.to_datetime(dollar_price['Date'])

In [6]:
print(SHEET_NAMES)

['NATIONAL', 'ABIA', 'ABUJA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA', 'CROSS RIVER', 'DELTA', 'RIVERS', 'EDO', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'BENUE', 'TARABA', 'YOBE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'ONDO', 'OGUN_', 'OSUN', 'OYO', 'JIGAWA_', 'KADUNA_', 'KANO_', 'KATSINA', 'KEBBI_', 'ZAMFARA_', 'SOKOTO']


In [7]:
SHEET_NAMES.remove('NATIONAL')

In [8]:
states =['ABIA', 'ABUJA', 'ANAMBRA', 'EBONYI', 'ENUGU', 'IMO', 'AKWA IBOM', 'BAYELSA',
         'CROSS RIVER', 'DELTA', 'RIVERS', 'EDO', 'ADAMAWA', 'BAUCHI', 'BORNO', 'GOMBE', 'BENUE', 'TARABA',
         'YOBE', 'KOGI', 'KWARA', 'NASSARAWA', 'NIGER', 'PLATEAU', 'EKITI', 'LAGOS', 'ONDO', 
         'OGUN', 'OSUN', 'OYO', 'JIGAWA', 'KADUNA', 'KANO', 'KATSINA', 'KEBBI', 'ZAMFARA', 'SOKOTO']

In [9]:
# data preprocessing
def get_data(dict_data, state_list):
    # get data sheet
    df = [get_sheet_data(dict_data, state) for state in SHEET_NAMES]
    # clean data
    df = [clean(state) for state in df]
    # reset index and rename to 'Date'
    for state in df:
        state.index =  pd.to_datetime(state.index)
        state.reset_index(inplace= True)
        state.rename(columns={'index':'Date'}, inplace= True)
    # melt data to 3 columns, 'Date', 'Food_items', and 'Food_prices'
    df = [state.melt(id_vars=['Date'], var_name="Food_items", value_name="Food_prices") for state in df]
    # add state location column for each dataframe in df
    for index, _ in enumerate(df):
        for state in state_list:
            if index == state_list.index(state):
                df[index]['Location'] = state
    # concatenate dataframes in df
    df = pd.concat(df)
    # merge with the dollar_rate dataframe
    df = pd.merge(df, dollar_price, left_on= 'Date', right_on= 'Date')

    return df
                
    

In [10]:
df = get_data(dict_data, states)
print(df.shape)
df.head()

(55599, 5)


,Date,Food_items,Food_prices,Location,Dollar Rate
0,2019-01-01,Agric eggs medium size1 Dozen,482.5,ABIA,363.3959
1,2019-01-01,Agric eggs medium size (1)1 Piece,46.35621,ABIA,363.3959
2,2019-01-01,"Beans brown,sold loose1 KG",489.583333,ABIA,363.3959
3,2019-01-01,Beans:white black eye. sold loose1 KG,482.77027,ABIA,363.3959
4,2019-01-01,Beef Bone in1 KG,987.675156,ABIA,363.3959


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55599 entries, 0 to 55598
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         55599 non-null  datetime64[ns]
 1   Food_items   55599 non-null  object        
 2   Food_prices  55598 non-null  object        
 3   Location     55599 non-null  object        
 4   Dollar Rate  55599 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 2.5+ MB


In [12]:
df = df.drop_duplicates()

In [13]:
df.isnull().sum()

Date           0
Food_items     0
Food_prices    1
Location       0
Dollar Rate    0
dtype: int64

In [14]:
# dropping NAN values
df.dropna(inplace=True)

In [15]:
df.dtypes

Date           datetime64[ns]
Food_items             object
Food_prices            object
Location               object
Dollar Rate           float64
dtype: object

In [16]:
# cleaning Food_prices column and converting it to float data type
def fix(x):
    try:
        value=float(x)
    except:
        try:
            number=x.split(' ')
            value= number[0]+'.'+number[1]
            value= float(value)
        except:
            value=0
    return value

In [17]:
df['Food_prices'] = df['Food_prices'].apply(fix)

In [18]:
df.dtypes

Date           datetime64[ns]
Food_items             object
Food_prices           float64
Location               object
Dollar Rate           float64
dtype: object

In [19]:
# Excluding rows that contains zero as Food_prices as Food Items cannot cost Zero Naira
df = df.loc[df['Food_prices'] != 0]

In [20]:
df.shape

(55595, 5)

In [21]:
# replacing Food price that seems extravagant with a more reasonable value: 'Check data_visualization.ipynb'
df.replace(to_replace=11400.000000, value= 1140, inplace=True)

In [22]:
df.describe()

,Food_prices,Dollar Rate
count,55595.000000,55595.000000
mean,653.329060,380.406726
std,534.922420,19.101255
min,31.052632,347.709600
25%,241.544058,361.785300
50%,450.450450,383.419500
75%,960.123050,390.281700
max,3553.532182,411.575100


In [23]:
# spliting the 'Date' column into readable ones for regression modelling
df['YEAR'] = df['Date'].dt.year
df['MONTH'] = df['Date'].dt.month
df['DAY'] = df['Date'].dt.day
df.drop(['Date'], axis = 'columns', inplace = True)

In [24]:
df.head()

,Food_items,Food_prices,Location,Dollar Rate,YEAR,MONTH,DAY
0,Agric eggs medium size1 Dozen,482.500000,ABIA,363.3959,2019,1,1
1,Agric eggs medium size (1)1 Piece,46.356210,ABIA,363.3959,2019,1,1
2,"Beans brown,sold loose1 KG",489.583333,ABIA,363.3959,2019,1,1
3,Beans:white black eye. sold loose1 KG,482.770270,ABIA,363.3959,2019,1,1
4,Beef Bone in1 KG,987.675156,ABIA,363.3959,2019,1,1


In [36]:
#imports for modelling
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from category_encoders import OneHotEncoder
from sklearn.pipeline import  make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [27]:
# train-test-split
train = df[df['YEAR'].isin([2019, 2020])]
test = df[df['YEAR'].isin([2021])]

In [28]:
train.shape, test.shape

((38095, 7), (17500, 7))

In [29]:
X_train = train.drop(['Food_prices'], axis = 1)
y_train = train['Food_prices']
X_test = test.drop(['Food_prices'], axis = 1)
y_test = test['Food_prices']

# Modelling

In [82]:
lin_model = make_pipeline(OneHotEncoder(use_cat_names=True), LinearRegression())

In [83]:
lin_model.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Food_items', 'Location'],
                               use_cat_names=True)),
                ('linearregression', LinearRegression())])

In [84]:

predx= lin_model.predict(X_train)
pred= lin_model.predict(X_test)
print(mean_absolute_error(y_test, pred))
print(np.sqrt(mean_squared_error(y_test, pred)))
print(r2_score(y_train, predx))
print(r2_score(y_test, pred))

138.69237495234898
218.2788911387724
0.938652489812856
0.8548851664617805


In [86]:
rfg_model = make_pipeline(OneHotEncoder(use_cat_names=True), RandomForestRegressor(random_state=42))

In [87]:
rfg_model.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Food_items', 'Location'],
                               use_cat_names=True)),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [ ]:

predx= rfg_model.predict(X_train)
pred= rfg_model.predict(X_test)
print(mean_absolute_error(y_test, pred))
print(np.sqrt(mean_squared_error(y_test, pred)))
print(r2_score(y_train, predx))
print(r2_score(y_test, pred))

118.32984376080236
185.31368627505756
0.999347378020121
0.8954068312567685


In [ ]:
xgb = make_pipeline(  OneHotEncoder(use_cat_names=True), XGBRegressor(random_state=42, learning_rate=0.6))

In [ ]:
xgb.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Food_items', 'Location'],
                               use_cat_names=True)),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.6,
                              max_delta_step=0, max_depth=6, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=8, num_parallel_tree=1,
                              predictor='auto', random_state=42, reg_alpha=0,
                              reg_lambda=1, scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=

In [ ]:
predx= xgb.predict(X_train)
pred= xgb.predict(X_test)
print(mean_absolute_error(y_test, pred))
print(np.sqrt(mean_squared_error(y_test, pred)))
print(r2_score(y_train, predx))
print(r2_score(y_test, pred))

121.52865665094451
188.7099990160613
0.9924751813871243
0.8915378635072635


In [72]:
ctb = make_pipeline(  OneHotEncoder(use_cat_names=True), CatBoostRegressor(random_state=42, learning_rate=0.5, depth=10))

In [73]:
ctb.fit(X_train, y_train)

0:	learn: 344.5144665	total: 21ms	remaining: 20.9s
1:	learn: 259.9037262	total: 44.8ms	remaining: 22.3s
2:	learn: 228.1417692	total: 65.4ms	remaining: 21.7s
3:	learn: 199.4111362	total: 86.3ms	remaining: 21.5s
4:	learn: 182.1013395	total: 108ms	remaining: 21.5s
5:	learn: 168.8574717	total: 128ms	remaining: 21.2s
6:	learn: 159.6028546	total: 149ms	remaining: 21.2s
7:	learn: 151.2150527	total: 170ms	remaining: 21.1s
8:	learn: 143.5288890	total: 191ms	remaining: 21s
9:	learn: 137.2523216	total: 212ms	remaining: 21s
10:	learn: 132.0498835	total: 235ms	remaining: 21.1s
11:	learn: 128.2841512	total: 258ms	remaining: 21.2s
12:	learn: 123.8961557	total: 284ms	remaining: 21.6s
13:	learn: 119.0150899	total: 310ms	remaining: 21.8s
14:	learn: 116.0048553	total: 340ms	remaining: 22.3s
15:	learn: 113.4127791	total: 363ms	remaining: 22.3s
16:	learn: 109.5706826	total: 385ms	remaining: 22.3s
17:	learn: 107.0544728	total: 406ms	remaining: 22.2s
18:	learn: 103.8811539	total: 428ms	remaining: 22.1s
19:	l

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Food_items', 'Location'],
                               use_cat_names=True)),
                ('catboostregressor',
                 <catboost.core.CatBoostRegressor object at 0x00000259C5B05700>)])

In [69]:
predx= ctb.predict(X_train)
pred= ctb.predict(X_test)
print(mean_absolute_error(y_test, pred))
print(np.sqrt(mean_squared_error(y_test, pred)))
print(r2_score(y_train, predx))
print(r2_score(y_test, pred))

115.4637101304
181.40671202179644
0.9992247324667248
0.8997706223054943


The Best performing model is the CatBoostRegressor, Let's save it.

In [88]:
import pickle

In [89]:
filename = 'finalized_model.sav'
pickle.dump(ctb, open(filename, 'wb'))

# Constraints
- A Time Series Model i.e ARIMA should have been used for this project but, the dataset contained just 35 Observations per Food item per Location which does not meet the basic requirement for Time Series analysis, this can be worked on to enable the use of a Time Series model and we can then compare performance with the CatBoostRegressor
- Also the dataset contained 43 independent Food Items in 36 different Location, using a Time Series model would require us having multiples models which would not be easy to handle. 